# The Price is Right

## Part C Order of Play

File 1: Modal.com and SpecialistAgent  
File 2: RAG, FrontierAgent, Ensemble Agent  
File 3: ScannerAgent, MessengerAgent   
File 4: AutonomousPlannerAgent  
File 5: The Price Is Right Finale


Now it's time for the Planning Agent

In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv
from agents.scanner_agent import ScannerAgent
import chromadb
import logging
load_dotenv(override=True)
openai = OpenAI()
MODEL = "gpt-5.1"

## Start with some test data

In [2]:
test_results = ScannerAgent().test_scan()
test_results

DealSelection(deals=[Deal(product_description="The Hisense R6 Series 55R6030N is a 55-inch 4K UHD Roku Smart TV that offers stunning picture quality with 3840x2160 resolution. It features Dolby Vision HDR and HDR10 compatibility, ensuring a vibrant and dynamic viewing experience. The TV runs on Roku's operating system, allowing easy access to streaming services and voice control compatibility with Google Assistant and Alexa. With three HDMI ports available, connecting multiple devices is simple and efficient.", price=178.0, url='https://www.dealnews.com/products/Hisense/Hisense-R6-Series-55-R6030-N-55-4-K-UHD-Roku-Smart-TV/484824.html?iref=rss-c142'), Deal(product_description='The Poly Studio P21 is a 21.5-inch LED personal meeting display designed specifically for remote work and video conferencing. With a native resolution of 1080p, it provides crystal-clear video quality, featuring a privacy shutter and stereo speakers. This display includes a 1080p webcam with manual pan, tilt, and

## Now let's create 3 pretend functions..

In [3]:
def scan_the_internet_for_bargains() -> str:
    """ This tool scans the internet for great deals and gets a curated list of promising deals """
    print("Fake function to scan the internet - this returns a hardcoded set of deals")
    return test_results.model_dump_json()

In [4]:
def estimate_true_value(description: str) -> str:
    """
    This tool estimates the true value of a product based on a text description of it
    """
    print(f"Fake function to estimating true value of {description[:20]}... - this always returns $300")
    return f"Product {description} has an estimated true value of $300"

In [5]:
def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float, url: str) -> str:
    """
    This tool notifies the user of a great deal, given a description of it, the price of the deal, and the estimated true value
    """
    print(f"Fake function to notify user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return "notification sent ok"

### Let's try them out

In [6]:
notify_user_of_deal("a new iphone", 100, 1000, "https://www.apple.com/iphone")

Fake function to notify user of a new iphone which costs 100 and estimate is 1000


'notification sent ok'

### OK now a big block of JSON

In [7]:
scan_function = {
        "name": "scan_the_internet_for_bargains",
        "description": "Returns top bargains scraped from the internet along with the price each item is being offered for",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": False
        }
    }

estimate_function = {
    "name": "estimate_true_value",
    "description": "Given the description of an item, estimate how much it is actually worth",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item to be estimated"
            },
        },
        "required": ["description"],
        "additionalProperties": False
    }
}

notify_function = {
    "name": "notify_user_of_deal",
    "description": "Send the user a push notification about the single most compelling deal; only call this one time",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item itself scraped from the internet"
            },
            "deal_price": {
                "type": "number",
                "description": "The price offered by this deal scraped from the internet"
            }
            ,
            "estimated_true_value": {
                "type": "number",
                "description": "The estimated actual value that this is worth"
            }
            ,
            "url": {
                "type": "string",
                "description": "The URL of this deal as scraped from the internet"
            }
        },
        "required": ["description", "deal_price", "estimated_true_value", "url"],
        "additionalProperties": False
    }
}

In [8]:
tools = [{"type": "function", "function": scan_function},
 {"type": "function", "function": estimate_function},
 {"type": "function", "function": notify_function}
 ]

In [9]:
tools

[{'type': 'function',
  'function': {'name': 'scan_the_internet_for_bargains',
   'description': 'Returns top bargains scraped from the internet along with the price each item is being offered for',
   'parameters': {'type': 'object',
    'properties': {},
    'required': [],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'estimate_true_value',
   'description': 'Given the description of an item, estimate how much it is actually worth',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string',
      'description': 'The description of the item to be estimated'}},
    'required': ['description'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'notify_user_of_deal',
   'description': 'Send the user a push notification about the single most compelling deal; only call this one time',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string',
      'description

In [10]:
def handle_tool_call(message):
    """
    Actually call the tools associated with this message
    """
    results = []
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [11]:
system_message = "You find great deals on bargain products using your tools, and notify the user of the best bargain."
user_message = """
First, use your tool to scan the internet for bargain deals. Then for each deal, use your tool to estimate its true value.
Then pick the single most compelling deal where the price is much lower than the estimated true value, and use your tool to notify the user.
Then just reply OK to indicate success.
"""
messages = [{"role": "system", "content": system_message},{"role": "user", "content": user_message}]

In [12]:
messages

[{'role': 'system',
  'content': 'You find great deals on bargain products using your tools, and notify the user of the best bargain.'},
 {'role': 'user',
  'content': '\nFirst, use your tool to scan the internet for bargain deals. Then for each deal, use your tool to estimate its true value.\nThen pick the single most compelling deal where the price is much lower than the estimated true value, and use your tool to notify the user.\nThen just reply OK to indicate success.\n'}]

In [13]:
done = False
while not done:
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        results = handle_tool_call(message)
        messages.append(message)
        messages.extend(results)
    else:
        done = True
response.choices[0].message.content

Fake function to scan the internet - this returns a hardcoded set of deals
Fake function to estimating true value of The Hisense R6 Serie... - this always returns $300
Fake function to estimating true value of The Poly Studio P21 ... - this always returns $300
Fake function to estimating true value of The Lenovo IdeaPad S... - this always returns $300
Fake function to estimating true value of The Dell G15 gaming ... - this always returns $300
Fake function to notify user of The Poly Studio P21 is a 21.5-inch LED personal meeting display designed specifically for remote work and video conferencing. With a native resolution of 1080p, it provides crystal-clear video quality, featuring a privacy shutter and stereo speakers. This display includes a 1080p webcam with manual pan, tilt, and zoom control, along with an ambient light sensor to adjust the vanity lighting as needed. It also supports 5W wireless charging for mobile devices, making it an all-in-one solution for home offices. which c

'OK'

## And now.. into an Autonomous Planning Agent

And switching the fake functions for REAL functions!

In [14]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [15]:
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [16]:
from agents.autonomous_planning_agent import AutonomousPlanningAgent
agent = AutonomousPlanningAgent(collection)

INFO:datasets:PyTorch version 2.9.1 available.
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Ensemble Agent] Initializing Ensemble Agent
INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready
INFO:root:[Neural Network Agent] Neural Network Agent is initializing
INFO:root:Neural Network is using cpu
INFO:root:[Neural Network Agent] Neural Network Agent is ready and weights are loaded
INFO:root:[Ensemble Agent] Ensemble Agent is ready
INFO:root:[

In [17]:
agent.plan()

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is kicking off a run
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is calling scanner
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 30 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
15:41:41 - LiteLLM:INFO: utils.py:3443 - 
LiteLLM completion() model= llama3.2; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3.2; provider = ollama
15:42:09 - LiteLLM:INFO: utils.py:1311 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:roo

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $899.00
INFO:root:[Neural Network Agent] Neural Network Agent is starting a prediction
INFO:root:[Neural Network Agent] Neural Network Agent completed - predicting $86.21
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $757.72
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
15:42:43 - LiteLLM:INFO: utils.py:3443 - 
LiteLLM completion() model= llama3.2; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3.2; provider = ollama
15:43:02 - LiteLLM:INFO: utils.py:1311 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensemble A

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $1899.00
INFO:root:[Neural Network Agent] Neural Network Agent is starting a prediction
INFO:root:[Neural Network Agent] Neural Network Agent completed - predicting $210.48
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $1590.25
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
15:43:09 - LiteLLM:INFO: utils.py:3443 - 
LiteLLM completion() model= llama3.2; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3.2; provider = ollama
15:43:23 - LiteLLM:INFO: utils.py:1311 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensembl

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $64.99
INFO:root:[Neural Network Agent] Neural Network Agent is starting a prediction
INFO:root:[Neural Network Agent] Neural Network Agent completed - predicting $100.59
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $73.05
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
15:43:50 - LiteLLM:INFO: utils.py:3443 - 
LiteLLM completion() model= llama3.2; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3.2; provider = ollama
15:44:07 - LiteLLM:INFO: utils.py:1311 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensemble Ag

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $649.99
INFO:root:[Neural Network Agent] Neural Network Agent is starting a prediction
INFO:root:[Neural Network Agent] Neural Network Agent completed - predicting $401.46
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $610.14
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is notifying user
INFO:root:[Messaging Agent] Messaging Agent is using Claude to craft the message
15:44:15 - LiteLLM:INFO: utils.py:3443 - 
LiteLLM completion() model= anthropic/claude-sonnet-4; provider = openrouter
INFO:LiteLLM:
LiteLLM completion() model= anthropic/claude-sonnet-4; provider = openrouter
15:44:18 - LiteLLM:INFO: utils.py:1311 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling succe

Opportunity(deal=Deal(product_description="Apple Mac mini M4 (2024) desktop with Apple M4 chip and Apple Intelligence, configured with 16GB of RAM and a 256GB SSD. This compact desktop runs macOS and is intended for general computing, creative work, and media tasks, offering Apple's latest on‑device performance improvements in a small, quiet chassis. Model identifier MU9D3LL/A.", price=400.0, url='https://www.dealnews.com/products/Apple/Apple-Mac-mini-M4-Desktop-2024-w-256-GB-SSD/484139.html?iref=rss-c39'), estimate=757.7210906982422, discount=357.72109069824216)